O objetivo deste notebook é ajustar os dados para serem processados no pycaret.

In [1]:
# Importando as bibliotecas 

import pandas as pd
import numpy as np
import pyarrow.parquet as pq
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import dask.dataframe as dd
import datetime
from datetime import timedelta



In [2]:
# Abrindo o parquet que tem os dados agregados

file = pd.read_parquet('agregator.parquet')
file

,recording_time,data_voo,phaseOfFlight-1,message0418DAA-1,message0422DAA-1,amscHprsovDrivF-1a,amscHprsovDrivF-1b,amscHprsovDrivF-2b,amscPrsovDrivF-1a,amscPrsovDrivF-1b,...,bleedOnStatus-2b,bleedOverpressCas-2a,bleedOverpressCas-2b,bleedPrecoolDiffPress-1a,bleedPrecoolDiffPress-1b,bleedPrecoolDiffPress-2a,bleedPrecoolDiffPress-2b,bleedPrsovClPosStatus-1a,bleedPrsovClPosStatus-2a,bleedPrsovFbk-1a
__null_dask_index__,,,,,,,,,,,,,,,,,,,,,
0,0,NaT,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN
1,50,NaT,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,...,NaN,0.0,0.0,0.03125,0.03125,0.03125,0.03125,NaN,NaN,1.750
2,100,NaT,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN
3,150,2022-12-06,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,200,NaT,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11744,587200,2023-01-18,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11746,587300,NaT,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,...,NaN,0.0,0.0,0.00000,-0.15625,0.06250,0.06250,NaN,NaN,1.875
11750,587500,NaT,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,...,NaN,0.0,0.0,0.12500,-0.15625,0.06250,0.06250,NaN,NaN,1.875


### Ajeitando a coluna de data

In [3]:
# Dropando as primeiras colunas
df = file.drop([0, 1, 2])
df.reset_index(drop=True, inplace=True)
df

,recording_time,data_voo,phaseOfFlight-1,message0418DAA-1,message0422DAA-1,amscHprsovDrivF-1a,amscHprsovDrivF-1b,amscHprsovDrivF-2b,amscPrsovDrivF-1a,amscPrsovDrivF-1b,...,bleedOnStatus-2b,bleedOverpressCas-2a,bleedOverpressCas-2b,bleedPrecoolDiffPress-1a,bleedPrecoolDiffPress-1b,bleedPrecoolDiffPress-2a,bleedPrecoolDiffPress-2b,bleedPrsovClPosStatus-1a,bleedPrsovClPosStatus-2a,bleedPrsovFbk-1a
0,150,2022-12-06,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,200,NaT,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN
2,250,NaT,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,...,NaN,0.0,0.0,0.03125,0.03125,0.03125,0.03125,NaN,NaN,1.750
3,300,NaT,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN
4,350,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1711563,587200,2023-01-18,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1711564,587300,NaT,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,...,NaN,0.0,0.0,0.00000,-0.15625,0.06250,0.06250,NaN,NaN,1.875
1711565,587500,NaT,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,...,NaN,0.0,0.0,0.12500,-0.15625,0.06250,0.06250,NaN,NaN,1.875
1711566,587700,NaT,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,...,NaN,0.0,0.0,0.12500,-0.15625,0.03125,0.03125,NaN,NaN,1.875


In [4]:
# Preenchendo a coluna de data vazia 
df['data_voo'] = pd.to_datetime(df['data_voo'])
last_valid_datetime = None

# Neste loop vamos preenchendo todas as colunas de data com as últimas datas 
for index, row in df.iterrows():
    if pd.isnull(row['data_voo']):
        # Checa se é NaN e coloca a última data válida
        df.at[index, 'data_voo'] = last_valid_datetime
    else:
        # Se não for NaN atualiza a data válida
        last_valid_datetime = row['data_voo']


In [5]:
df

,recording_time,data_voo,phaseOfFlight-1,message0418DAA-1,message0422DAA-1,amscHprsovDrivF-1a,amscHprsovDrivF-1b,amscHprsovDrivF-2b,amscPrsovDrivF-1a,amscPrsovDrivF-1b,...,bleedOnStatus-2b,bleedOverpressCas-2a,bleedOverpressCas-2b,bleedPrecoolDiffPress-1a,bleedPrecoolDiffPress-1b,bleedPrecoolDiffPress-2a,bleedPrecoolDiffPress-2b,bleedPrsovClPosStatus-1a,bleedPrsovClPosStatus-2a,bleedPrsovFbk-1a
0,150,2022-12-06,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,200,2022-12-06,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN
2,250,2022-12-06,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,...,NaN,0.0,0.0,0.03125,0.03125,0.03125,0.03125,NaN,NaN,1.750
3,300,2022-12-06,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN
4,350,2022-12-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1711563,587200,2023-01-18,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1711564,587300,2023-01-18,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,...,NaN,0.0,0.0,0.00000,-0.15625,0.06250,0.06250,NaN,NaN,1.875
1711565,587500,2023-01-18,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,...,NaN,0.0,0.0,0.12500,-0.15625,0.06250,0.06250,NaN,NaN,1.875
1711566,587700,2023-01-18,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,...,NaN,0.0,0.0,0.12500,-0.15625,0.03125,0.03125,NaN,NaN,1.875


In [6]:
df_fixed_date = df
df_fixed_date

,recording_time,data_voo,phaseOfFlight-1,message0418DAA-1,message0422DAA-1,amscHprsovDrivF-1a,amscHprsovDrivF-1b,amscHprsovDrivF-2b,amscPrsovDrivF-1a,amscPrsovDrivF-1b,...,bleedOnStatus-2b,bleedOverpressCas-2a,bleedOverpressCas-2b,bleedPrecoolDiffPress-1a,bleedPrecoolDiffPress-1b,bleedPrecoolDiffPress-2a,bleedPrecoolDiffPress-2b,bleedPrsovClPosStatus-1a,bleedPrsovClPosStatus-2a,bleedPrsovFbk-1a
0,150,2022-12-06,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,200,2022-12-06,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN
2,250,2022-12-06,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,...,NaN,0.0,0.0,0.03125,0.03125,0.03125,0.03125,NaN,NaN,1.750
3,300,2022-12-06,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN
4,350,2022-12-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1711563,587200,2023-01-18,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1711564,587300,2023-01-18,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,...,NaN,0.0,0.0,0.00000,-0.15625,0.06250,0.06250,NaN,NaN,1.875
1711565,587500,2023-01-18,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,...,NaN,0.0,0.0,0.12500,-0.15625,0.06250,0.06250,NaN,NaN,1.875
1711566,587700,2023-01-18,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,...,NaN,0.0,0.0,0.12500,-0.15625,0.03125,0.03125,NaN,NaN,1.875


### Criando a coluna de falha semanal 

O objetivo dessa função é criar uma flag semanal. Assim, quando houve uma falha a última semana inteira é marcado como falha 

In [9]:
# Criando a coluna de falha semanal 
df_fixed_date['failure_week'] = 0
df_fixed_date

,recording_time,data_voo,phaseOfFlight-1,message0418DAA-1,message0422DAA-1,amscHprsovDrivF-1a,amscHprsovDrivF-1b,amscHprsovDrivF-2b,amscPrsovDrivF-1a,amscPrsovDrivF-1b,...,bleedOverpressCas-2a,bleedOverpressCas-2b,bleedPrecoolDiffPress-1a,bleedPrecoolDiffPress-1b,bleedPrecoolDiffPress-2a,bleedPrecoolDiffPress-2b,bleedPrsovClPosStatus-1a,bleedPrsovClPosStatus-2a,bleedPrsovFbk-1a,failure_week
0,150,2022-12-06,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,200,2022-12-06,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,0
2,250,2022-12-06,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.03125,0.03125,0.03125,0.03125,NaN,NaN,1.750,0
3,300,2022-12-06,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,0
4,350,2022-12-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1711563,587200,2023-01-18,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1711564,587300,2023-01-18,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00000,-0.15625,0.06250,0.06250,NaN,NaN,1.875,0
1711565,587500,2023-01-18,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.12500,-0.15625,0.06250,0.06250,NaN,NaN,1.875,0
1711566,587700,2023-01-18,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.12500,-0.15625,0.03125,0.03125,NaN,NaN,1.875,0


In [11]:
# Convertendo a coluna de string para datetime
df_fixed_date['data_voo'] = pd.to_datetime(df_fixed_date['data_voo'])


In [12]:
# Obtendo todas as datas que houveram falhas

# Falhas do motor 1 
get_failure_422 = df_fixed_date[(df_fixed_date['message0422DAA-1'] != 0) & (df_fixed_date['message0422DAA-1'].notna())]
# Falhas do motor 2 
get_failure_418 = df_fixed_date[(df_fixed_date['message0418DAA-1'] != 0) & (df_fixed_date['message0418DAA-1'].notna())]


In [13]:
# Checando os valores onde houveram falhas 
finish_week = get_failure_422['data_voo'].unique()
day_of_fails_1 = get_failure_418['data_voo'].unique()
print(finish_week)
print(day_of_fails_1)


<DatetimeArray>
['2022-12-09 00:00:00']
Length: 1, dtype: datetime64[ns]
<DatetimeArray>
[]
Length: 0, dtype: datetime64[ns]


In [14]:
# Criando o datetime da data onde há falha
finish_week = pd.to_datetime(finish_week)
finish_week

<DatetimeArray>
['2022-12-09 00:00:00']
Length: 1, dtype: datetime64[ns]

In [15]:
# Definindo as datas para criar o intervalo de falhas semanais
start_datetime = pd.to_datetime('2022-12-02')  
end_datetime = pd.to_datetime('2022-12-09')

In [16]:
# Criando o dataframe que tem todas as falhas que se encaixam no intervalo de uma semana
conditional = (df_fixed_date['data_voo'] >= start_datetime ) & (df_fixed_date['data_voo'] <= end_datetime )

In [17]:
# Define como 1 todos os valores que encaixam na condicional
df_fixed_date.loc[conditional, 'failure_week'] = 1

In [18]:
# Checando se a falha ocorreu no dia d e a função funcionou corretamente
d = datetime.date(2022, 12, 2)
test_function = df_fixed_date[df_fixed_date['data_voo'].dt.date == d]
test_function

,recording_time,data_voo,phaseOfFlight-1,message0418DAA-1,message0422DAA-1,amscHprsovDrivF-1a,amscHprsovDrivF-1b,amscHprsovDrivF-2b,amscPrsovDrivF-1a,amscPrsovDrivF-1b,...,bleedOverpressCas-2a,bleedOverpressCas-2b,bleedPrecoolDiffPress-1a,bleedPrecoolDiffPress-1b,bleedPrecoolDiffPress-2a,bleedPrecoolDiffPress-2b,bleedPrsovClPosStatus-1a,bleedPrsovClPosStatus-2a,bleedPrsovFbk-1a,failure_week
5999,150,2022-12-02,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
6000,250,2022-12-02,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.03125,0.03125,0.03125,-128.00000,NaN,NaN,2.0,1.0
6001,450,2022-12-02,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.03125,0.03125,0.03125,-128.00000,NaN,NaN,2.0,1.0
6002,650,2022-12-02,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.03125,0.03125,0.03125,-128.00000,NaN,NaN,2.0,1.0
6003,850,2022-12-02,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.03125,0.03125,0.03125,-128.00000,NaN,NaN,2.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16924,514800,2022-12-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
16925,514850,2022-12-02,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,1.0
16926,514900,2022-12-02,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.03125,0.03125,0.03125,0.03125,NaN,NaN,2.0,1.0
16927,514950,2022-12-02,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,1.0


In [22]:
# Vendo todas as colunas
pd.set_option('display.max_columns', None)


In [23]:
df_weekly_fail = df_fixed_date
df_weekly_fail.describe()

,recording_time,data_voo,phaseOfFlight-1,message0418DAA-1,message0422DAA-1,amscHprsovDrivF-1a,amscHprsovDrivF-1b,amscHprsovDrivF-2b,amscPrsovDrivF-1a,amscPrsovDrivF-1b,amscPrsovDrivF-2b,basBleedLowPressF-1a,basBleedLowPressF-2b,basBleedLowTempF-1a,basBleedLowTempF-2b,basBleedOverPressF-1a,basBleedOverPressF-2b,basBleedOverTempF-1a,basBleedOverTempF-2b,bleedFavTmCmd-1a,bleedFavTmCmd-1b,bleedFavTmCmd-2a,bleedFavTmCmd-2b,bleedFavTmFbk-1a,bleedFavTmFbk-1b,bleedFavTmFbk-2b,bleedHprsovCmdStatus-1a,bleedHprsovCmdStatus-1b,bleedHprsovCmdStatus-2a,bleedHprsovCmdStatus-2b,bleedHprsovOpPosStatus-1a,bleedHprsovOpPosStatus-1b,bleedHprsovOpPosStatus-2a,bleedHprsovOpPosStatus-2b,bleedMonPress-1a,bleedMonPress-1b,bleedMonPress-2a,bleedMonPress-2b,bleedOnStatus-1a,bleedOnStatus-1b,bleedOnStatus-2b,bleedOverpressCas-2a,bleedOverpressCas-2b,bleedPrecoolDiffPress-1a,bleedPrecoolDiffPress-1b,bleedPrecoolDiffPress-2a,bleedPrecoolDiffPress-2b,bleedPrsovClPosStatus-1a,bleedPrsovClPosStatus-2a,bleedPrsovFbk-1a,failure_week
count,1.711568e+06,1711568,577239.000000,83885.0,83885.000000,1006640.0,1006640.0,1.006640e+06,1006640.0,1006640.0,1006640.0,1006640.0,1006640.0,1006640.0,1006640.0,1006640.0,1006640.0,1006640.0,1.006640e+06,1.006640e+06,1.006640e+06,1.006640e+06,1.006640e+06,1.006640e+06,1.006640e+06,1.006640e+06,577239.000000,577239.000000,577239.000000,577239.000000,577239.000000,577239.000000,577239.000000,577239.000000,1.006640e+06,1.006640e+06,1.006640e+06,1.006640e+06,577239.000000,577239.000000,577239.000000,1006640.0,1006640.0,1.006640e+06,1.006640e+06,1.006640e+06,1.006640e+06,577239.000000,577239.000000,1.006640e+06,1.711568e+06
mean,3.790895e+06,2023-01-01 09:19:42.696568320,0.812275,0.0,0.059784,0.0,0.0,1.192085e-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.573273e-03,3.180158e+01,6.593341e+01,2.855849e+01,2.807024e+01,3.329375e+01,6.837823e+01,2.935101e+01,0.281157,0.329692,0.265256,0.265259,0.618747,0.618548,0.578972,0.578985,6.407141e+01,6.398840e+01,6.332182e+01,6.283305e+01,0.869378,0.869376,0.827411,0.0,0.0,5.797311e-01,5.800782e-01,1.238804e-01,1.882997e-03,0.130629,0.161955,4.484847e+01,1.106728e-01
min,1.500000e+02,2022-12-02 00:00:00,0.000000,0.0,0.000000,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,-5.120000e+02,-5.120000e+02,2.500000e-01,0.000000e+00,-5.120000e+02,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.750000e-01,2.500000e-01,-5.120000e+02,-5.120000e+02,0.000000,0.000000,0.000000,0.0,0.0,-2.500000e-01,-2.500000e-01,-1.280000e+02,-1.280000e+02,0.000000,0.000000,2.500000e-01,0.000000e+00
25%,1.561450e+06,2022-12-17 00:00:00,0.000000,0.0,0.000000,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,6.250000e-01,5.000000e-01,3.750000e-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.262500e+01,4.262500e+01,4.287500e+01,4.287500e+01,1.000000,1.000000,1.000000,0.0,0.0,4.375000e-01,4.375000e-01,3.750000e-01,3.750000e-01,0.000000,0.000000,7.500000e-01,0.000000e+00
50%,3.284525e+06,2023-01-04 00:00:00,1.000000,0.0,0.000000,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,9.500000e+01,0.000000e+00,0.000000e+00,6.250000e-01,9.875000e+01,3.750000e-01,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,4.950000e+01,4.937500e+01,5.025000e+01,5.025000e+01,1.000000,1.000000,1.000000,0.0,0.0,6.250000e-01,6.250000e-01,5.625000e-01,5.625000e-01,0.000000,0.000000,8.750000e-01,0.000000e+00
75%,5.650700e+06,2023-01-16 00:00:00,2.000000,0.0,0.000000,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,9.625000e+01,1.160000e+02,9.350000e+01,9.350000e+01,1.000000e+02,1.198750e+02,9.737500e+01,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,7.350000e+01,7.337500e+01,7.487500e+01,7.487500e+01,1.000000,1.000000,1.000000,0.0,0.0,7.500000e-01,7.500000e-01,6.562500e-

In [24]:
# Após o describe vamos remover todas as colunas que não fazem sentido para a série temporal 
empty_columns = ['bleedOverpressCas-2b','bleedOverpressCas-2a','basBleedOverTempF-1a','basBleedOverTempF-2b',
                 'amscPrsovDrivF-1a','amscPrsovDrivF-1b','amscPrsovDrivF-2b','basBleedLowPressF-1a',
                 'basBleedLowPressF-2b','basBleedLowTempF-1a','basBleedLowTempF-2b','basBleedOverPressF-1a',
                 'amscHprsovDrivF-1a','amscHprsovDrivF-1b']



In [26]:
# Aqui estão todas as colunas vazias 
df_weekly_fail = df_weekly_fail.drop(empty_columns, axis=1)
df_weekly_fail

,recording_time,data_voo,phaseOfFlight-1,message0418DAA-1,message0422DAA-1,amscHprsovDrivF-2b,basBleedOverPressF-2b,bleedFavTmCmd-1a,bleedFavTmCmd-1b,bleedFavTmCmd-2a,bleedFavTmCmd-2b,bleedFavTmFbk-1a,bleedFavTmFbk-1b,bleedFavTmFbk-2b,bleedHprsovCmdStatus-1a,bleedHprsovCmdStatus-1b,bleedHprsovCmdStatus-2a,bleedHprsovCmdStatus-2b,bleedHprsovOpPosStatus-1a,bleedHprsovOpPosStatus-1b,bleedHprsovOpPosStatus-2a,bleedHprsovOpPosStatus-2b,bleedMonPress-1a,bleedMonPress-1b,bleedMonPress-2a,bleedMonPress-2b,bleedOnStatus-1a,bleedOnStatus-1b,bleedOnStatus-2b,bleedPrecoolDiffPress-1a,bleedPrecoolDiffPress-1b,bleedPrecoolDiffPress-2a,bleedPrecoolDiffPress-2b,bleedPrsovClPosStatus-1a,bleedPrsovClPosStatus-2a,bleedPrsovFbk-1a,failure_week
0,150,2022-12-06,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,200,2022-12-06,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,1.0,1.0,NaN,1
2,250,2022-12-06,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,1.750,0.25,0.750,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.375,0.375,1.125,1.125,NaN,NaN,NaN,0.03125,0.03125,0.03125,0.03125,NaN,NaN,1.750,1
3,300,2022-12-06,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,1.0,1.0,NaN,1
4,350,2022-12-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1711563,587200,2023-01-18,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1711564,587300,2023-01-18,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,1.875,0.25,0.875,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44.250,44.125,44.500,44.500,NaN,NaN,NaN,0.00000,-0.15625,0.06250,0.06250,NaN,NaN,1.875,0
1711565,587500,2023-01-18,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,1.750,0.25,0.875,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44.250,44.125,44.500,44.500,NaN,NaN,NaN,0.12500,-0.15625,0.06250,0.06250,NaN,NaN,1.875,0
1711566,587700,2023-01-18,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,1.750,0.25,0.875,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44.250,44.125,44.375,44.375,NaN,NaN,NaN,0.12500,-0.15625,0.03125,0.03125,NaN,NaN,1.875,0


In [27]:
# Importando os dados como csv 
df_weekly_fail.to_csv('df_weekly_fail.csv', index=True)